## Lógica Computacional: 21/22## 
---
##Trabalho 1## 

$Grupo$ $7$ 

*   David José de Sousa Machado (A91665)
*   Ivo Miguel Gomes Lima (A90214)
---
#Inicialização

Para a resolução destes exercícios usamos a biblioteca [OR-Tools](https://developers.google.com/optimization) que criou uma interface para o SCIP. Esta biblioteca foi instalada com o commando `pip install ortools`.

In [2]:
!pip install ortools

In [7]:
import networkx as nx
from ortools.linear_solver import pywraplp
from tabulate import tabulate
import random

# Problema 1: Horário de uma *StartUp*

Foi pedida a criação de um horário semanal para uma *Startup*, seguindo as seguintes condições:

1. Cada reunião ocupa uma sala (enumeradas $1...S\,$) durante um *“slot”* $(\text{tempo},\text{dia})$.  Assume-se os dias enumerados $1..D$ e, em cada dia, os tempos enumerados $1..T$.

2.  Cada reunião tem associado um projeto (enumerados $1..P$) e um conjunto de participantes. Os diferentes colaboradores são enumerados $1..C$.

3. Cada projeto tem associado um conjunto de colaboradores, dos quais um  é o líder. Cada projeto realiza um dado número de reuniões semanais. São *“inputs”* do problema o conjunto de colaboradores de cada projeto, o seu líder e o número de reuniões semanais.

4. O líder do projeto participa em todas as reuniões do seu projeto; os restantes colaboradores podem ou não participar consoante a sua disponibilidade, num mínimo (*“quorum”*) de  $50\%$ do total de colaboradores do projeto.  A disponibilidade de cada participante, incluindo o lider,  é um conjunto de *“slots”* (*“inputs”* do problema).

**Análise do problema**

Para criarmos um horário coerente e compatível com as disponibilidades de cada um dos intervenientes, foi necessário estabelecer algumas restrições.

**Condições inerentes**

As condições inerentes são relativas à verificação da coerência do mesmo. Estas condições são:

*   Deve existir um número de reuniões semanais por projeto $R_p$, dado no input do problema:

$$\forall_{p<P} \quad \sum_{s<S,\ d<D,\ h<H,\ c_{Líder}} x_{p,s,d,h,c} = R_p$$

*   Para haver reunião de um projeto numa certa sala, dia e hora, o líder tem de estar presente:

$$\forall_{p<P} \quad \sum_{s<S,\ d<D,\ h<H,\ c_{Líder}} x_{p,s,d,h,c} = 1$$

>   **Nota:** As condições acima enunciadas acabam por culminar numa única pois o Líder tem de ir a todas as $R$ reuniões o que implica a existência das mesmas.

*   Não pode haver reunião de um projeto numa certa sala, dia e hora, se os colaboradores estiverem indisponíveis:

$$\forall_{s<S} \cdot \forall_{d<D} \cdot \forall_{h<H} \cdot \forall_{p<P} \quad \sum_{c\in Proj\ \land\ c \in\ Indisponível } x_{p,s,d,h,c} = 0\$$

*   Cada colaborador, numa dada sala, dia e hora, não pode participar num projeto que não é o seu:

$$\forall_{s<S} \cdot \forall_{d<D} \cdot \forall_{h<H} \cdot \forall_{p<P}\cdot \forall_{c\notin Proj } \quad \sum_{} x_{p,s,d,h,c} = 0 $$

**Limitações** (que impõem limites máximos à alocação)

*   Cada sala, num dado dia e hora, apenas pode acolher um projeto:

$$\forall_{s<S} \cdot \forall_{d<D}\cdot \forall_{h<H} \quad \sum_{p< P,\,c\in Proj } x_{p,s,d,h,c} \leq 1$$

*   Cada projeto só pode ter no máximo uma reunião por dia e hora:

$$\forall_{p < P} \cdot \forall_{d < D}\cdot \forall_{h < H}  \quad \sum_{s < S,\,c\in Proj } x_{p,s,d,h,c} \leq 1$$

*   Cada colaborador, num dado dia e hora, só pode participar na sala do seu projeto:

$$\forall_{s<S} \cdot \forall_{d<D}\cdot \forall_{h<H} \cdot \forall_{p<P} \cdot \forall_{c \in Proj} \quad \sum_{s< S} x_{p,s,d,h,c} \leq 1$$

>   **Nota:** Novamente as duas condições acima chegam a uma única pois ao garantirmos que cada colaborador apenas pode participar na reunião do seu projeto, garantimos que ela existe.

**Obrigações** (que impõem limites mínimos à alocação)

*   Para haver reunião de um projeto numa certa sala, dia e hora, o líder e pelo menos $50\%$ dos colaboradores devem estar disponíveis assim como a sala:

$$\forall_{s<S} \cdot \forall_{d<D}\cdot \forall_{h<H} \cdot \forall_{p<P} \quad\sum_{c\in\ (Proj\ \land\ Líder) } x_{p,s,d,h,c} \geq \frac{c}{2}$$

#Implementação:

**Gerador de Testes Aleatório**

In [5]:
Slots = [(d, h) for d in range(5) for h in range(8)]
Colabs = set(range(30))

Colaboradores = [random.sample(Slots, 20) for _ in range(30)]
Projectos = []
for _ in range(5):
  team = random.sample(Colabs, 6)
  Projectos.append((random.randint(1, 5), team))
  Colabs = Colabs - set(team)

for c in range(30):
  Colaboradores[c].sort()
  print(c, Colaboradores[c])

for num_r, workers in Projectos:
  print(num_r, workers)

0 [(0, 1), (0, 2), (0, 4), (0, 5), (1, 0), (1, 2), (1, 4), (1, 5), (1, 6), (1, 7), (2, 0), (2, 1), (2, 2), (2, 5), (2, 7), (3, 0), (3, 6), (4, 2), (4, 4), (4, 5)]
1 [(0, 0), (0, 5), (1, 0), (1, 1), (1, 3), (1, 6), (1, 7), (2, 1), (2, 3), (2, 4), (2, 7), (3, 0), (3, 1), (3, 2), (3, 3), (3, 7), (4, 0), (4, 2), (4, 3), (4, 7)]
2 [(0, 0), (0, 1), (0, 3), (0, 5), (1, 1), (1, 6), (1, 7), (2, 1), (2, 2), (2, 6), (2, 7), (3, 1), (3, 3), (3, 4), (3, 5), (3, 6), (3, 7), (4, 0), (4, 4), (4, 5)]
3 [(0, 0), (0, 1), (0, 4), (0, 6), (0, 7), (1, 2), (1, 3), (1, 4), (1, 7), (2, 1), (2, 2), (2, 4), (3, 1), (3, 2), (3, 3), (3, 4), (3, 5), (4, 0), (4, 4), (4, 5)]
4 [(0, 0), (0, 1), (0, 3), (0, 4), (0, 6), (0, 7), (1, 0), (1, 5), (1, 6), (2, 0), (2, 1), (2, 2), (2, 4), (2, 6), (2, 7), (3, 0), (3, 1), (3, 3), (4, 1), (4, 5)]
5 [(0, 0), (0, 1), (0, 2), (0, 6), (0, 7), (1, 1), (1, 2), (1, 6), (1, 7), (2, 0), (2, 1), (2, 2), (2, 7), (3, 0), (3, 1), (3, 2), (3, 7), (4, 0), (4, 3), (4, 6)]
6 [(0, 0), (0, 2), (0,

In [6]:
horario = pywraplp.Solver.CreateSolver('SCIP')

# Sala, Dias, Horas
S, D, H = 5, 5, 8
# Proj, Colab
P, C = 5, 30
# Num Reunioes Semanais
R = 5

# Inicialização
x = {}
for s in range(S):
  for d in range(D):
    for h in range(H):
      for p in range(P):
        for c in range(C):
          x[s, d, h, p, c] = horario.BoolVar('x[%i, %i, %i, %i, %i]' % (s, d, h, p, c))

# Condições inerentes

# O líder tem de estar em todas as R reuniões do seu projecto
for p in range(P):
  horario.Add(sum(x[s, d, h, p, Projectos[p][1][0]] for s in range(S) for d in range(D) for h in range(H)) == Projectos[p][0])

# Slot (d, h) fora da disponibilidade do colaborador, logo não pode ser usado
for s in range(S):
  for d in range(D):
    for h in range(H):
      for p in range(P):
        for c in range(C):
          if (d, h) not in Colaboradores[c]:
            horario.Add(x[s, d, h, p, c] == 0)

# Colaboradores que não são do projecto não podem estar nele
for s in range(S):
  for d in range(D):
    for h in range(H):
      for p in range(P):
        for c in range(C):
          if c not in Projectos[p][1]:
            horario.Add(x[s, d, h, p, c] == 0)

# Limitações

# Cada sala tem alocada, no máximo, um projeto
for s in range(S):
  for d in range(D):
    for h in range(H):
      horario.Add(sum(x[s, d, h, p, Projectos[p][1][0]] for p in range(P)) <= 1)

# Cada colaborador de um projeto só pode estar numa sala
for d in range(D):
  for h in range(H):
    for p in range(P):
      for c in Projectos[p][1]:
        horario.Add(sum(x[s, d, h, p, c]  for s in range(S)) <= 1)

# Obrigações

# Participação de 50% com o líder incluido
for s in range(S):
  for d in range(D):
    for h in range(H):
      for p in range(P):
        horario.Add(sum(x[s, d, h, p, c] for c in Projectos[p][1]) >= 3 * x[s, d, h, p, Projectos[p][1][0]])

# Fazer o solve
status = horario.Solve()
if status == pywraplp.Solver.OPTIMAL:
  for p in range(P):
    presencas = []
    for dia in range(D):
      print("DIA {:<14}".format(dia), end="")
    print()
    for hora in range(H):
      for dia in range(D):
        print("||", hora, end=" || ")
        for s in range(S):
          if round(x[s, dia, hora, p, Projectos[p][1][0]].solution_value()) == 1:
            presencas.append([c for c in range(C) if round(x[s, dia, hora, p, c].solution_value())])
            print(s, end=" ")
          else:
            print("x", end=" ")
      print()
    print(presencas)
else:
  print("impossível")

DIA 0             DIA 1             DIA 2             DIA 3             DIA 4             
|| 0 || x x x x x || 0 || x x x x x || 0 || x x x x x || 0 || x x x x x || 0 || x x x x x 
|| 1 || x x x x x || 1 || x x x x x || 1 || x x x x x || 1 || x x x x x || 1 || x x x x x 
|| 2 || x x x x x || 2 || x x x x x || 2 || x x x x x || 2 || x x x x x || 2 || x x x x x 
|| 3 || x x x x x || 3 || x x x x x || 3 || x x x x x || 3 || 0 x x x x || 3 || x x x x x 
|| 4 || x x x x x || 4 || x x x x x || 4 || x x x x x || 4 || x x x x x || 4 || x x x x x 
|| 5 || x x x x x || 5 || x x x x x || 5 || x x x x x || 5 || x x x x x || 5 || x x x x x 
|| 6 || x x x x x || 6 || x x x x x || 6 || x x x x x || 6 || x x x x x || 6 || x x x x x 
|| 7 || x x x x x || 7 || x x x x x || 7 || x x x x x || 7 || x x x x x || 7 || x x x x x 
[[2, 15, 26]]
DIA 0             DIA 1             DIA 2             DIA 3             DIA 4             
|| 0 || x x x x x || 0 || x x x x x || 0 || x x x x x || 0 || x x x x x || 0